In [ ]:
# ===============================
# Шаблон для регрессии (Цена)
# ===============================

# Импорт необходимых библиотек
import pandas as pd               # Для работы с табличными данными
import numpy as np                # Для математических операций
from xgboost import XGBRegressor   # XGBoost для регрессии
from catboost import CatBoostRegressor   # CatBoost для регрессии (отлично работает с категориальными данными)
from lightgbm import LGBMRegressor         # LightGBM для регрессии
from sklearn.metrics import mean_squared_error  # Для оценки качества (MSE)

# -------------------------------
# Загрузка и подготовка данных
# -------------------------------
# Укажите пути к вашим файлам (обычно предоставляются в соревновании)
train_data_path = '/kaggle/input/moscow/train (5).csv'
test_data_path  = '/kaggle/input/moscow/test (5).csv'

# Чтение данных
train_data = pd.read_csv(train_data_path)
test_data  = pd.read_csv(test_data_path)

# Предполагаем, что целевой столбец называется 'price'
# Разделяем данные: признаки (X) и целевая переменная (y)
X_train = train_data.drop('price', axis=1)
y_train = train_data['price']

# Если в тестовом наборе есть столбец 'price' (например, для локальной проверки), удаляем его,
# иначе просто используем все данные для предсказания.
if 'price' in test_data.columns:
    X_test = test_data.drop('price', axis=1)
    y_test = test_data['price']
else:
    X_test = test_data.copy()
    y_test = None

# -------------------------------
# Преобразование категориальных признаков
# -------------------------------
# Преобразуем все текстовые признаки в тип 'category'
for col in X_train.columns:
    if X_train[col].dtype == 'object':
        X_train[col] = X_train[col].astype('category')
        if col in X_test.columns:
            X_test[col] = X_test[col].astype('category')

# Для моделей CatBoost и XGBoost (с поддержкой категориальных данных) формируем список категориальных признаков
cat_features = X_train.select_dtypes(include=['category']).columns.tolist()

# -------------------------------
# Обучение моделей для регрессии
# -------------------------------

# 1. XGBoost для регрессии
model_xgb = XGBRegressor(
    n_estimators=1000,         # Количество деревьев
    learning_rate=0.1,         # Скорость обучения
    max_depth=6,               # Максимальная глубина деревьев
    verbosity=0,               # Подавление вывода
    enable_categorical=True    # Поддержка категориальных данных (экспериментально)
)
if y_test is not None:
    model_xgb.fit(X_train, y_train, 
                  eval_set=[(X_test, y_test)], 
                  early_stopping_rounds=50,
                  verbose=False)
else:
    model_xgb.fit(X_train, y_train, verbose=False)
y_pred_xgb = model_xgb.predict(X_test)

# 2. CatBoost для регрессии
model_cat = CatBoostRegressor(
    iterations=1000,    # Количество итераций (деревьев)
    learning_rate=0.1,  # Скорость обучения
    depth=6,            # Глубина деревьев
    verbose=0           # Подавление вывода
)
if y_test is not None:
    model_cat.fit(X_train, y_train, 
                  cat_features=cat_features,
                  eval_set=(X_test, y_test),
                  early_stopping_rounds=50,
                  verbose=False)
else:
    model_cat.fit(X_train, y_train, cat_features=cat_features, verbose=False)
y_pred_cat = model_cat.predict(X_test)

# 3. LightGBM для регрессии
model_lgb = LGBMRegressor(
    n_estimators=1000,  # Количество деревьев
    learning_rate=0.1,  # Скорость обучения
    max_depth=6         # Максимальная глубина
)
if y_test is not None:
    model_lgb.fit(X_train, y_train,
                  eval_set=[(X_test, y_test)],
                  early_stopping_rounds=50)
else:
    model_lgb.fit(X_train, y_train)
y_pred_lgb = model_lgb.predict(X_test)

# -------------------------------
# Ансамблирование предсказаний
# -------------------------------
# Итоговое предсказание рассчитывается как среднее арифметическое по трём моделям
y_pred_ensemble = (y_pred_xgb + y_pred_cat + y_pred_lgb) / 3

# Если имеются истинные значения, оцениваем качество (MSE)
if y_test is not None:
    mse = mean_squared_error(y_test, y_pred_ensemble)
    print("MSE регрессионного ансамбля: {:.2f}".format(mse))

# -------------------------------
# Сохранение предсказаний в CSV
# -------------------------------
submission = pd.DataFrame({'index': X_test.index, 'price': y_pred_ensemble})
submission.to_csv('submission_price.csv', index=False)
print("Предсказания цены сохранены в 'submission_price.csv'")


In [ ]:
# ===============================
# Шаблон для классификации (категориальный фактор)
# ===============================

import pandas as pd               # Работа с данными
import numpy as np                # Математические операции
from xgboost import XGBClassifier   # XGBoost для классификации
from catboost import CatBoostClassifier  # CatBoost для классификации
from lightgbm import LGBMClassifier        # LightGBM для классификации
from sklearn.metrics import accuracy_score  # Метрика для оценки точности

# ----- 1. Настройка: пути и имя целевого столбца -----
target_col = "area_class"  # Замените на имя столбца для классификации
train_data_path = '/path/to/your/train.csv'
test_data_path  = '/path/to/your/test.csv'

# ----- 2. Загрузка данных -----
train_data = pd.read_csv(train_data_path)
test_data  = pd.read_csv(test_data_path)

# Если целевой столбец ещё числовой, его можно преобразовать в категории.
# Например, разбив на 4 квантильных группы:
if pd.api.types.is_numeric_dtype(train_data[target_col]):
    num_bins = 4
    y_train = pd.qcut(train_data[target_col], q=num_bins, labels=False, duplicates='drop')
else:
    y_train = train_data[target_col]

# Формирование обучающих признаков
X_train = train_data.drop(target_col, axis=1)

# Обработка тестового набора: если целевой столбец присутствует, для локальной проверки
if target_col in test_data.columns:
    if pd.api.types.is_numeric_dtype(test_data[target_col]):
        y_test = pd.qcut(test_data[target_col], q=num_bins, labels=False, duplicates='drop')
    else:
        y_test = test_data[target_col]
    X_test = test_data.drop(target_col, axis=1)
else:
    X_test = test_data.copy()
    y_test = None

# ----- 3. Преобразование категориальных признаков -----
for col in X_train.columns:
    if X_train[col].dtype == 'object':
        X_train[col] = X_train[col].astype('category')
        if col in X_test.columns:
            X_test[col] = X_test[col].astype('category')
            
cat_features = X_train.select_dtypes(include=['category']).columns.tolist()

# ----- 4. Обучение моделей для классификации -----

# Модель XGBoost для классификации
model_xgb = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6,
    verbosity=0,
    enable_categorical=True
)
if y_test is not None:
    model_xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)],
                  early_stopping_rounds=50, verbose=False)
else:
    model_xgb.fit(X_train, y_train, verbose=False)
proba_xgb = model_xgb.predict_proba(X_test)

# Модель CatBoost для классификации
model_cat = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    verbose=0
)
if y_test is not None:
    model_cat.fit(X_train, y_train, cat_features=cat_features,
                  eval_set=(X_test, y_test),
                  early_stopping_rounds=50, verbose=False)
else:
    model_cat.fit(X_train, y_train, cat_features=cat_features, verbose=False)
proba_cat = model_cat.predict_proba(X_test)

# Модель LightGBM для классификации
model_lgb = LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6
)
if y_test is not None:
    model_lgb.fit(X_train, y_train, eval_set=[(X_test, y_test)],
                  early_stopping_rounds=50)
else:
    model_lgb.fit(X_train, y_train)
proba_lgb = model_lgb.predict_proba(X_test)

# ----- 5. Ансамблирование предсказаний -----
avg_proba = (proba_xgb + proba_cat + proba_lgb) / 3
y_pred_ensemble = np.argmax(avg_proba, axis=1)

# Если имеются истинные метки, вычисляем точность
if y_test is not None:
    acc = accuracy_score(y_test, y_pred_ensemble)
    print("Точность классификационного ансамбля: {:.2f}%".format(acc * 100))

# ----- 6. Сохранение предсказаний -----
submission = pd.DataFrame({'index': X_test.index, target_col: y_pred_ensemble})
submission.to_csv('submission_classification.csv', index=False)
print("Предсказания классификации сохранены в 'submission_classification.csv'")


In [ ]:
# ===============================
# Шаблон для регрессии (числовой фактор)
# ===============================

import pandas as pd               # Работа с данными
import numpy as np                # Математические операции
from xgboost import XGBRegressor   # XGBoost для регрессии
from catboost import CatBoostRegressor   # CatBoost для регрессии
from lightgbm import LGBMRegressor         # LightGBM для регрессии
from sklearn.metrics import mean_squared_error  # Метрика для оценки качества

# ----- 1. Настройка: пути и имя целевого столбца -----
target_col = "price"  # Замените на имя столбца, который нужно предсказать (например, "price", "area", и т.д.)
train_data_path = '/kaggle/input/moscow/train (5).csv'
test_data_path  = '/kaggle/input/moscow/test (5).csv'

# ----- 2. Загрузка данных -----
train_data = pd.read_csv(train_data_path)
test_data  = pd.read_csv(test_data_path)

# Формирование обучающей выборки: удаляем столбец target и сохраняем его отдельно
X_train = train_data.drop(target_col, axis=1)
y_train = train_data[target_col]

# Для тестового набора: если целевой столбец присутствует (локальная проверка), удаляем его
if target_col in test_data.columns:
    X_test = test_data.drop(target_col, axis=1)
    y_test = test_data[target_col]
else:
    X_test = test_data.copy()
    y_test = None

# ----- 3. Преобразование категориальных признаков -----
# Преобразуем все текстовые признаки в тип category
for col in X_train.columns:
    if X_train[col].dtype == 'object':
        X_train[col] = X_train[col].astype('category')
        if col in X_test.columns:
            X_test[col] = X_test[col].astype('category')

# Составляем список категориальных признаков для CatBoost и XGBoost
cat_features = X_train.select_dtypes(include=['category']).columns.tolist()

# ----- 4. Обучение моделей -----

# Модель XGBoost для регрессии
model_xgb = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6,
    verbosity=0,
    enable_categorical=True
)
if y_test is not None:
    model_xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)],
                  early_stopping_rounds=50, verbose=False)
else:
    model_xgb.fit(X_train, y_train, verbose=False)
y_pred_xgb = model_xgb.predict(X_test)

# Модель CatBoost для регрессии
model_cat = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    verbose=0
)
if y_test is not None:
    model_cat.fit(X_train, y_train, cat_features=cat_features,
                  eval_set=(X_test, y_test),
                  early_stopping_rounds=50, verbose=False)
else:
    model_cat.fit(X_train, y_train, cat_features=cat_features, verbose=False)
y_pred_cat = model_cat.predict(X_test)

# Модель LightGBM для регрессии
model_lgb = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6
)
if y_test is not None:
    model_lgb.fit(X_train, y_train, eval_set=[(X_test, y_test)],
                  early_stopping_rounds=50)
else:
    model_lgb.fit(X_train, y_train)
y_pred_lgb = model_lgb.predict(X_test)

# ----- 5. Ансамблирование предсказаний -----
y_pred_ensemble = (y_pred_xgb + y_pred_cat + y_pred_lgb) / 3

# Если есть истинные значения, вычисляем MSE
if y_test is not None:
    mse = mean_squared_error(y_test, y_pred_ensemble)
    print("MSE регрессионного ансамбля: {:.2f}".format(mse))

# ----- 6. Сохранение предсказаний -----
submission = pd.DataFrame({'index': X_test.index, target_col: y_pred_ensemble})
submission.to_csv('submission_regression.csv', index=False)
print("Предсказания сохранены в 'submission_regression.csv'")
